This notebooks builds tables forthe labeling application based on the provided Excel data file.
Though the tables should be created from the original sources

The following tables will be created:


**licensing_documents**


| Field |  Type |  Comment |
|-------|-------|----------|
| doc_id | varchar(64) | |
| content | text | |
| license_sents | text | Automatically Extracted? Which one should be displayed? |
| key_sents | text |Automatically Extracted? Which one should be displayed?|


**licensing_doc_company**

| Field |  Type |  Comment |
|-------|-------|----------|
| doc_id | varchar(64) | |
| company_id | varchar(64) | Normalized/cleaned company name. E.g. “CEREGENE” for “Ceregene, Inc.” |
| company_name | varchar(64) | Name to be displayed on the form. Needs to match the text document. E.g. “Ceregene, Inc.” |


**licensing_assignments**

| Field |  Type |  Comment |
|-------|-------|----------|
| doc_id | varchar(64) | |
| user_id | varchar(64) | This should be a unique ID that users use to start the survey. E.g. username of the email address.|
| assignment_id | varchar(128) | doc_id + '^' + user_id |


# Setup and Database Connection

In [2]:
import os
import json
import sys
import numpy as np
import pandas as pd
import sqlalchemy as sa

In [45]:
# load database credentials and change database to "datalabeling"
db_creds = json.load(open(f"/home/{os.environ['USER']}/.arc_config/databases/sqlserver-2-3306.txt"))
connection_uri = '/'.join( db_creds['default_uri'].split('/')[:3] + ['datalabeling'])

# create connection
conn = sa.engine.create_engine(connection_uri)

# test if connection works
q = """
SELECT NOW() AS jetzt
"""
display(pd.read_sql(q, conn))

,jetzt
0,2021-03-28 14:53:56


In [106]:
def drop_table(tn):
    try:
        conn.execute(f"""DROP TABLE {tn}""")
    except:
        pass
    
def describe_table(tn):
    q = f"""
    DESCRIBE {tn}
    """
    display(pd.read_sql(q, conn))
    
def show_tables():
    q = """
    SHOW TABLES
    """
    display(pd.read_sql(q, conn))

# Load Data from Excel

In [92]:
df = pd.read_excel('data/Sample_LicensingLabelling.xlsx')
print(f"Number of records: {df.shape[0]:,}")
df.head()

Number of records: 26


,Unnamed: 0,date,doc_id,content,licensing_agreement(combined_model_preds),license_sents,key_sents,New_Org_list,Checked_by,licensing_agreement_check,license_type,licensor,licensee,Rule_flag_Batch
0,16272,2004-04-19 19:58:00,PRN0000020040420e04j00001,Alliance Pharmaceutical Corp. and IL YANG Phar...,1,We are very pleased to have IL YANG as our exc...,"""We are very pleased to have IL YANG as our ex...","['YANG', 'CEO', 'IL', 'Alliance', '[]', Oxygen...",Hard,1,regular,Alliance,IL YANG,2
1,16377,2004-04-30 08:00:00,BWR0000020040430e04u0033g,Symantec and Clearswift Settle Pending Litigat...,1,"As part of the agreement, Clearswift obtained ...","As part of the agreement, Clearswift obtained ...","[Clearswift','Symantec','Clearswift"", 'Symante...",Natali,1,Regular,Symantec,Clearswift,10
2,16387,2004-04-29 15:14:00,PRN0000020040429e04t00env,Trans-Lux Sells Australian Subsidiary To Paltr...,1,The sale provides Paltronics Australasia with ...,"In conjunction with the sale, Trans-Lux grante...","[Paltronics Australasia','Trans-Lux"", 'Paltron...",Achal,1,Exclusive,Paltronics Australasia,Trans-Lux',1
3,16488,2004-05-12 20:43:00,BWR0000020040513e05d000gp,Maxim Announces Partial Summary Judgment in Li...,1,The partial summary judgment decision confirms...,The decision concludes that the subject licens...,"['Qualcomm', 'Maxim', 'Court', Qualcomm','Cour...",Hard,1,regular,CDMA,Qualcomm,2
4,16563,2004-05-19 16:06:00,BWR0000020040519e05j009hp,Corixa and the Australian Nuclear Science and ...,1,"Under the terms of the agreement, ANSTO will p...","SEATTLE & SYDNEY, Australia - (BUSINESS WIRE) ...","['Corixa', ANSTO','Corixa','Corixa"", 'ANSTO']""",Achal,1,Regular,ANSTO,Corixa,1


# Table: licensing_documents

In [108]:
import html
table_name = 'licensing_documents'

In [109]:
df2 = df[['doc_id', 'content', 'license_sents', 'key_sents']].copy()
for col in ['content', 'license_sents', 'key_sents']:
    df2[col] = df2[col].map(lambda x: html.escape(x) if pd.notnull(x) else x)
# df2.set_index('doc_id', inplace=True)

display(df2.head())

## Upload
drop_table(table_name)
df2.to_sql(table_name, con=conn, index=None,
          dtype={
              'doc_id': sa.types.VARCHAR(length=64),
              'content': sa.types.TEXT,
              'license_sents': sa.types.TEXT,
              'key_sents': sa.types.TEXT
          })

describe_table(table_name)

,doc_id,content,license_sents,key_sents
0,PRN0000020040420e04j00001,Alliance Pharmaceutical Corp. and IL YANG Phar...,We are very pleased to have IL YANG as our exc...,&quot;We are very pleased to have IL YANG as o...
1,BWR0000020040430e04u0033g,Symantec and Clearswift Settle Pending Litigat...,"As part of the agreement, Clearswift obtained ...","As part of the agreement, Clearswift obtained ..."
2,PRN0000020040429e04t00env,Trans-Lux Sells Australian Subsidiary To Paltr...,The sale provides Paltronics Australasia with ...,"In conjunction with the sale, Trans-Lux grante..."
3,BWR0000020040513e05d000gp,Maxim Announces Partial Summary Judgment in Li...,The partial summary judgment decision confirms...,The decision concludes that the subject licens...
4,BWR0000020040519e05j009hp,Corixa and the Australian Nuclear Science and ...,"Under the terms of the agreement, ANSTO will p...","SEATTLE &amp; SYDNEY, Australia - (BUSINESS WI..."


,Field,Type,Null,Key,Default,Extra
0,doc_id,varchar(64),YES,,None,
1,content,text,YES,,None,
2,license_sents,text,YES,,None,
3,key_sents,text,YES,,None,


#  Table: licensing_doc_company

processing the "New_Org_list" column in the Excel sheet. There should be a better data source.

In [110]:
import re


In [113]:
docs = []
company = []
for i, row in df.iterrows():
    dd = row.doc_id
    
    for x in row.New_Org_list.split(','):
        x1 = x.replace("'", "").replace('"', "").replace('[', "").replace(']', "").strip()
        if len(x1)>0:
            docs.append(dd)
            company.append(x1)
            
comp_df = pd.DataFrame({'doc_id': docs, 'company_name': company})
comp_df['company_id'] = comp_df['company_name'].map(lambda s: re.sub('\W', '_', s.upper()))

print(f"Number of records: {comp_df.shape[0]:,}")

display(comp_df.head(10))

## Upload
table_name = 'licensing_doc_company'
drop_table(table_name)
comp_df.to_sql(table_name, con=conn, index=None,
          dtype={
              'doc_id': sa.types.VARCHAR(length=64),
              'company_name': sa.VARCHAR(length=64),
              'company_id': sa.VARCHAR(length=64),
          })

describe_table(table_name)

Number of records: 137


,doc_id,company_name,company_id
0,PRN0000020040420e04j00001,YANG,YANG
1,PRN0000020040420e04j00001,CEO,CEO
2,PRN0000020040420e04j00001,IL,IL
3,PRN0000020040420e04j00001,Alliance,ALLIANCE
4,PRN0000020040420e04j00001,Oxygent,OXYGENT
5,PRN0000020040420e04j00001,Alliance,ALLIANCE
6,PRN0000020040420e04j00001,IL YANG,IL_YANG
7,PRN0000020040420e04j00001,Oxygent,OXYGENT
8,BWR0000020040430e04u0033g,Clearswift,CLEARSWIFT
9,BWR0000020040430e04u0033g,Symantec,SYMANTEC


,Field,Type,Null,Key,Default,Extra
0,doc_id,varchar(64),YES,,None,
1,company_name,varchar(64),YES,,None,
2,company_id,varchar(64),YES,,None,


# Table: licensing_assignments

In [114]:
assignment_df = df2[['doc_id']].copy()
assignment_df['user_id'] = 'pmolnar'
assignment_df['assignment_id'] = assignment_df.apply(lambda r: f"{r.doc_id}^{r.user_id}", axis=1)
display(assignment_df.head())

table_name = 'licensing_assignments'
drop_table(table_name)
assignment_df.to_sql(table_name, con=conn, index=None,
                    dtype= {
                        'doc_id': sa.types.VARCHAR(length=64),
                        'user_id': sa.types.VARCHAR(length=64),
                        'assignment_id': sa.types.VARCHAR(length=129),
                    })
describe_table(table_name)

,doc_id,user_id,assignment_id
0,PRN0000020040420e04j00001,pmolnar,PRN0000020040420e04j00001^pmolnar
1,BWR0000020040430e04u0033g,pmolnar,BWR0000020040430e04u0033g^pmolnar
2,PRN0000020040429e04t00env,pmolnar,PRN0000020040429e04t00env^pmolnar
3,BWR0000020040513e05d000gp,pmolnar,BWR0000020040513e05d000gp^pmolnar
4,BWR0000020040519e05j009hp,pmolnar,BWR0000020040519e05j009hp^pmolnar


,Field,Type,Null,Key,Default,Extra
0,doc_id,varchar(64),YES,,None,
1,user_id,varchar(64),YES,,None,
2,assignment_id,varchar(129),YES,,None,
